In [ ]:
import numpy as np
import json
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization

# Running this notebook requires  the following files and folders:
#     1. "./train_matrix.npy"
#     2. "./test_matrix.npy"
#     3. "./full_matrix.npy"
#     4. "./All_Trained_Models/"          *folder to store trained models
#     5. "./Top_20_Models/"               *folder to store trained top 20 models

# Running this notebook outputs the following:
#     1. "./All_Trained_Models/"                         *with all the trained models inside this folder
#     2. "./All_Trained_Models/Result_Score.json"
#     3. "./All_Trained_Models/Result_Store_Sorted.json"
#     4. "./Top_20_Models/"                               *with the top 20 trained models inside this folder

In [ ]:
# gets the input_matrix for training from file
input_matrix = np.load('./train_matrix.npy')

# Prepare the input and target matrix set for training
# Target set matrix is just the input set matrix shifted by one on the time-series axis
input_set = input_matrix[:,:-1,:,:,:]
target_set = input_matrix[:,1:,:,:,:]

In [ ]:
# Split the time series data into batches according to day to test out results of different batches
# ie, train the model on 1 day = 1 batch or 2 day = 1 batch

# This is done in order to iterate and get the best number of days split to produce the best model performance
# Model might not train well if the batch sequence length is too short or too long
# 61 days is too long to train the LSTM with

# If not completely divisible by the days, the remainder time-series data are discarded at the back

def generate_batch(input_set,target_set,day_in_batch=1,matrix=input_set):
    time_series_batch = int(day_in_batch*24*4)
    if day_in_batch == 61:
        time_series_batch = time_series_batch - 1
    total_df_time_series = matrix.shape[1]
    batch_num = int(total_df_time_series/time_series_batch)
    input_set_batch = np.zeros((batch_num,time_series_batch,36,46,1))
    target_set_batch = np.zeros((batch_num,time_series_batch,36,46,1))

    for i in range(batch_num):
        input_set_batch[i] = input_set[:,(time_series_batch)*(i):(time_series_batch)*(i+1),:,:,:]
        target_set_batch[i] = target_set[:,(time_series_batch)*(i):(time_series_batch)*(i+1),:,:,:]
    
    return input_set_batch, target_set_batch, batch_num

In [ ]:
# Define a function to enable the creation of ConvLSTM model while looping over different parameters
def Seq_Model(filter_size=1,layer=1,dropout=0,recurrent_dropout=0):
    
    seq = Sequential()

    for i in range(layer):
        seq.add(ConvLSTM2D(filters=filter_size, kernel_size=(3, 3),
                           input_shape=(None, 36, 46, 1),padding='same', 
                           dropout = dropout, 
                           recurrent_dropout = recurrent_dropout,
                           return_sequences=True))
        seq.add(BatchNormalization())

    seq.add(Conv3D(filters=1, kernel_size=(1, 1, 1), activation='sigmoid',padding='same', data_format='channels_last'))
    seq.compile(loss='mean_squared_error', optimizer='RMSprop')
    
    return seq

In [ ]:
# Train the network
model_filter = [40,60,80]
model_layer = [2]
day_split = [1,2]
epoch = 100
dropout = [0, 0.1]
recurrent_dropout = [0,0.5,0.8]

for filter_size in model_filter:
    for layer_depth in model_layer:            
        for i,j in enumerate(day_split):
            for dropout_num in dropout:
                for recurrent_dropout_num in recurrent_dropout:
                # Initialise Model
                    seq = Seq_Model(filter_size=filter_size,layer=layer_depth,dropout=dropout_num,recurrent_dropout=recurrent_dropout_num)

                    # Model Training
                    input_set_batch, target_set_batch,batch_num = generate_batch(input_set,target_set,day_in_batch=j,matrix=input_set)
                    print("Training for {} filter {} layer {} day {} dropout {} rdropout model".format(filter_size,layer_depth,j,dropout_num,recurrent_dropout_num))
                    History = seq.fit(input_set_batch[:batch_num], target_set_batch[:batch_num], batch_size=1,
                        epochs=epoch, validation_split=0.1)

                    # save model History
                    history_dict = History.history
                    json.dump(history_dict, open("./All_Trained_Models/{}filter{}layer{}day{}drop{}rdrop_model_history.json".format(filter_size,layer_depth,j,dropout_num,recurrent_dropout_num), 'w'))

                    # serialize model to JSON
                    model_json = seq.to_json()
                    with open("./All_Trained_Models/{}filter{}layer{}day{}drop{}rdrop_model.json".format(filter_size,layer_depth,j,dropout_num,recurrent_dropout_num), "w") as json_file:
                        json_file.write(model_json)

                    # serialize weights to HDF5
                    seq.save_weights("./All_Trained_Models/{}filter{}layer{}day{}drop{}rdrop_model.h5".format(filter_size,layer_depth,j,dropout_num,recurrent_dropout_num))
                    print("Saved {}filter{}layer{}day{}drop{}rdrop_model to disk".format(filter_size,layer_depth,j,dropout_num,recurrent_dropout_num))

# Evaluate the RMSE of all the trained models rigorously

In [ ]:
# gets the input_matrix for training from file
input_matrix = np.load('./test_matrix.npy')

In [ ]:
def get_rmse(T5_actual,T5_pred): 
    return np.sqrt(np.mean((T5_actual-T5_pred)**2))

In [ ]:
# Baseline RMSE is the RMSE you get when using the given demand values at T to be values for (T+1 to T+5) 
# This is the minimum RMSE to beat, as a model is considered useless if just blindly using the last seen values can outperform 
# the model prediction

# The parameters here should be the same as from the training parameters, 
# as the program will loop through the file to find the models with the given parameters
# will have error if file cannot be found.
model_layers = [2]
days_split = [1,2]
model_filter = [40,60,80]
dropout = [0, 0.1]
rdropout = [0, 0.5, 0.8]

T_shift = [0,48,56,76,96,112,130,144,156,187]   # 0 - 187, shifts the test data set window
Day_Feed = [14]         # Predict T+5 from x days to test the model prediction across multiple sequences and get the average
Overall_min = 1               # Initialise the min RMSE
Result_Store = {}

# Loops through the files in "./All_Trained_Models/" and evaluate the model prediction RMSE performance using test data
for model_layer in model_layers:
    for i,model_day in enumerate(days_split):
        for j,filters in enumerate(model_filter):
            for dropout_num in dropout:
                for rdropout_num in rdropout:
                    # load json and create model
                    json_file = open("./All_Trained_Models/{}filter{}layer{}day{}drop{}rdrop_model.json".format(filters,model_layer,model_day,dropout_num,rdropout_num), 'r')
                    loaded_model_json = json_file.read()
                    json_file.close()
                    loaded_model = model_from_json(loaded_model_json)
                    # load weights into new model
                    loaded_model.load_weights(("./All_Trained_Models/{}filter{}layer{}day{}drop{}rdrop_model.h5".format(filters,model_layer,model_day,dropout_num,rdropout_num)))
                    # print("Loaded model from disk")

                    # 
                    RMSE_Day_total = 0
                    RMSE_Day_total_baseline = 0 # for baseline RMSE----------------
                    Max_Day_rmse = 0
                    Min_Day_rmse = 1

                    Day_all_store = {}
                    for day in Day_Feed:
                        RMSE_T_shift_total = 0
                        Max_T_shift_rmse = 0
                        Min_T_shift_rmse = 1
                        day_feed = day*24*4

                        for a in T_shift:
                            input_set = input_matrix[:, a : a + day_feed,:,:,:]
                            target_set = input_matrix[:, a : a + day_feed + 5,:,:,:]
                            train_14_day = input_set[0][:,:,:,:]

                            for k in range(5):
                                new_pred = loaded_model.predict(train_14_day[np.newaxis, ::, ::, ::, ::])
                                new = new_pred[::, -1, ::, ::, ::]
                                train_14_day = np.concatenate((train_14_day, new), axis=0)

                            T5_pred = train_14_day[-5:,:,:,:]
                            T5_actual = target_set[0][-5:,:,:,:]                    
                            rmse = get_rmse(T5_actual,T5_pred)  

                            # ----------------------------for baseline RMSE----------------
                            T5_baseline = train_14_day[-6:-5,:,:,:]
                            baseline = train_14_day[-6:-5,:,:,:]
                            for k in range(4):                        
                                T5_baseline = np.concatenate((T5_baseline, baseline), axis=0)
                            baseline_rmse = get_rmse(T5_actual,T5_baseline)  
                            # ----------------------------for baseline RMSE----------------

                            if rmse > Max_T_shift_rmse:
                                Max_T_shift_rmse = rmse

                            if rmse < Min_T_shift_rmse:
                                Min_T_shift_rmse = rmse                      

                            if rmse > Max_Day_rmse:
                                Max_Day_rmse = rmse

                            if rmse < Min_Day_rmse:
                                Min_Day_rmse = rmse
                                filter_min = filters
                                layer_min = model_layer
                                day_min = model_day

                            RMSE_T_shift_total += rmse
                            RMSE_Day_total += rmse    
                            RMSE_Day_total_baseline += baseline_rmse  # for baseline RMSE----------------
                        RMSE_T_shift_avg = RMSE_T_shift_total/len(T_shift)

                        Day_store = {}
                        Day_store['Avg'] = RMSE_T_shift_avg
                        Day_store['Max'] = Max_T_shift_rmse
                        Day_store['Min'] = Min_T_shift_rmse

                        Day_all_store[day] = Day_store

                        Result_Store['{}filter{}layer{}day{}drop{}rdrop'.format(filters,model_layer,model_day,dropout_num,rdropout_num)] = Day_all_store
                    RMSE_Day_total_avg = RMSE_Day_total/(len(T_shift)*len(Day_Feed))
                    RMSE_Day_total_baseline_avg = RMSE_Day_total_baseline/(len(T_shift)*len(Day_Feed)) # for baseline RMSE----------------

                    Model_store = Day_all_store
                    Model_store['Avg'] = RMSE_Day_total_avg
                    Model_store['Max'] = Max_Day_rmse
                    Model_store['Min'] = Min_Day_rmse            

                    Result_Store['{}filter{}layer{}day{}drop{}rdrop'.format(filters,model_layer,model_day,dropout_num,rdropout_num)] = Model_store  

                    print('Average RMSE for {}filter{}layer{}day{}drop{}rdrop_model is {}. Baseline: {}'.format(filters,model_layer,model_day,dropout_num,rdropout_num,RMSE_Day_total_avg,RMSE_Day_total_baseline_avg))

# save Result_Store to JSON
json.dump(Result_Store, open("./All_Trained_Models/Result_Score.json", 'w'))          

In [ ]:
# load Result_Store
Result_Store = json.load(open("./All_Trained_Models/Result_Score.json", 'r')) 

Result_Store_Sorted = {}
i = 1
for key, value in sorted(Result_Store.items(), key=lambda tup: (tup[1]['Avg'])):
    Model_dict = {}
    Model_dict['Model'] = key
    Model_dict['Avg'] = value['Avg']
    Model_dict['Max'] = value['Max']
    Model_dict['Min'] = value['Min']
    Result_Store_Sorted['{}'.format(i)] = Model_dict  
    i+=1

json.dump(Result_Store_Sorted, open("./All_Trained_Models/Result_Store_Sorted.json", 'w'))

print(Result_Store_Sorted)

# Select the Top 20 Models and train on the whole dataset (61 days)

In [ ]:
# gets the input_matrix for training from file
input_matrix = np.load('./full_matrix.npy')

# Prepare the input and target matrix set for training
# Target set matrix is just the input set matrix shifted by one on the time-series axis
input_set = input_matrix[:,:-1,:,:,:]
target_set = input_matrix[:,1:,:,:,:]

In [ ]:
file = open("./All_Trained_Models/Result_Store_Sorted.json", 'r')
sorted_model = json.loads(file.read())

for i in range(20):
    i = i + 1
    s = sorted_model[str(i)]['Model']
    
    #extract the 'day' value from the file title
    day = float(s[s.find('layer')+len('layer'):s.rfind('day')])
    
    # load json and create model
    json_file = open("./All_Trained_Models/{}_model.json".format(sorted_model[str(i)]['Model']), 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    
    # load weights into model
    loaded_model.load_weights(("./All_Trained_Models/{}_model.h5".format(sorted_model[str(i)]['Model'])))
    # print("Loaded model from disk")

    input_set_batch, target_set_batch,batch_num = generate_batch(input_set,target_set,day_in_batch=day,matrix=input_set)
    loaded_model.compile(loss='mean_squared_error', optimizer='RMSprop')
    History = loaded_model.fit(input_set_batch[:batch_num], target_set_batch[:batch_num], batch_size=1, epochs=100, validation_split=0.1)
    
    # save model History
    history_dict = History.history
    json.dump(history_dict, open("./Top_20_Models/{}_model_history.json".format(sorted_model[str(i)]['Model']), 'w'))

    # serialize model to JSON
    model_json = loaded_model.to_json()
    with open("./Top_20_Models/{}_model.json".format(sorted_model[str(i)]['Model']), "w") as json_file:
        json_file.write(model_json)

    # serialize weights to HDF5
    loaded_model.save_weights("./Top_20_Models/{}_model.h5".format(sorted_model[str(i)]['Model']))
    print("Saved {}_model to Top_20_Models Folder".format(sorted_model[str(i)]['Model']))   
    
    